In [1]:
import pandas as pd
import numpy as np
from matrixprofile import *
from matrixprofile.discords import discords
from matplotlib import pyplot as plt
from scipy.io import arff
from binarytree import Node
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.feature_selection import mutual_info_classif
from scipy.stats import entropy
from math import log, e
import pydotplus
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import train_test_split
from sklearn import tree
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score, f1_score, classification_report
from sklearn.metrics import roc_curve, auc, roc_auc_score

In [2]:
#VERSIONE CON USO DEI MOTIF,DISCORD O ENTRAMBI E CON RIMOZIONE DEI CANDIDATI GIA SCELTI

In [3]:
def retrieve_all(Ts,window_size): #fornita la Ts calcola e restituisce mp, motifs, motifs_distances e discords
    Ts=Ts[0:127] #rimuovo l'attributo "classe"

    dfMP = pd.DataFrame(Ts).astype(float) # genero Dframe per lavorarci su, DA CAPIRE PERCHE SERVE FLOAT
    mp, mpi = matrixProfile.stomp(dfMP[0].values,window_size) #OK STOMP

    #PREPARO TUPLA DA PASSARE ALLA FUN MOTIF (RICHIEDE TUPLA FATTA DA MP E MPI)
    tupla=mp,mpi

    mot, motif_dist  =motifs.motifs(dfMP[0].values,tupla,2)

    #CALCOLO MOTIFS
   # print('Motifs starting position: '+str(mot)+ ' Motifs values (min distances): '+str(motif_dist))
   # print(" ")

    #CALCOLO DISCORDS
    dis= discords(mp,window_size,2)
    #print('Discords starting position: '+str(dis))
    
    tupla=mp,mot,motif_dist,dis
    return tupla

In [4]:
#ogni motif e identificato da almeno due indici di partenza nella Ts, ne prendo uno solo rappresentativo
#genero poi struttura contenente gli indici di partenza di tutti i candidati

def candidateFilter(CandidatesList): 
    counterNumberMotif=0
    counterNumberDiscord=0
    l2=np.array([])
    for i in range (len(CandidatesList['Motif'])): #per ogni entry (per ogni record)
        numMotif=len(CandidatesList['Motif'].iloc[i])
        numDiscord=len(CandidatesList['Discord'].iloc[i])
        counterNumberDiscord+=numDiscord
        #print(numMotif)
        for j in range (numMotif): # per ogni lista di motif
            l1=CandidatesList['Motif'].iloc[i] #prima lista
            l2=np.append(l2,l1[j][0]) #prendo primo valore di ogni lista
            counterNumberMotif+=1
            
    
        CandidatesList['Motif'].iloc[i]=l2
        l2=np.array([]) #svuoto array
    
    return CandidatesList,counterNumberMotif,counterNumberDiscord

In [5]:
#
def buildCandidatesUsedList(CandidatesList,numberOfMotif,numberOfDiscord):
    CandidatesUsedList=pd.DataFrame(columns=['Used'],index=range(0,numberOfMotif+numberOfDiscord))
    boolList=[False] * (numberOfMotif+numberOfDiscord)
    CandidatesUsedList['Used']= boolList
    return CandidatesUsedList
                                    

In [6]:
def getDataStructures(df,window_size):
    #trasformo da stringa a numero il campo target 
    le = LabelEncoder()
    num_classes = le.fit_transform(df['target'])
    df['target']=num_classes
    df['TsIndex']=np.arange(len(df))
    window_size=5
    #diz={'Motif':[],'Motif-Dist':[],'Discord':[]}
    diz={'Motif':[],'Discord':[]}

    #CALCOLO MOTIF E DISCORD E LI INSERISCO NEL DIZIONARIO
    for i in range(len(df)):
        Ts = np.array(df.iloc[i][0:-2].values)
        mp,mot,motif_dist,dis = retrieve_all(Ts,window_size)
        diz['Motif'].insert(i, mot)
        diz['Discord'].insert(i, dis)

    #GENERO DFRAME DA DIZIONARIO

    CandidatesList = pd.DataFrame(diz)
    CandidatesList,numberOfMotif,numberOfDiscord=candidateFilter(CandidatesList)
    CandidatesUsedList=buildCandidatesUsedList(CandidatesList,numberOfMotif,numberOfDiscord)

    print('Candidati estratti')
    #print(MotifsList['Motif'])
    print(CandidatesList)
    print(numberOfMotif,numberOfDiscord)
    print(CandidatesUsedList)
    
    return mp,CandidatesList,numberOfMotif,numberOfDiscord,CandidatesUsedList

In [7]:
#per ogni Ts calcolo Dprofile con ogni candidato e inserisco la distanza minima con candidato i-esimo nella colonna i-esima
def computeSubSeqDistance(dataset,CandidatesList):
    
    #quantifico il num di candidati e in base a tale valore genero colonne per dfForDTree
    numberOfCandidates=0
    for i in range(len(CandidatesList)):
            numberOfCandidates+=len(CandidatesList['Motif'].loc[i])
            numberOfCandidates+=len(CandidatesList['Discord'].loc[i])
    columnsList=np.arange(numberOfCandidates)
    columnsList2=list()
    lastAttribute=['TsIndex','class']
    prefix='cand'
    for i in columnsList:
        columnsList2.append(prefix+str(i)) 
    columnsList2.append('TsIndex')
    columnsList2.append('class')
    dfForDTree=pd.DataFrame(columns=columnsList2,index=range(0,len(dataset)))

    #per ogni Ts, scandisco ogni candidato e calcolo la distanza minore 
    for i in range(len(dataset)):
        #acquisisco la Ts
        TsToCompare = np.array(dataset.iloc[i].values) 
        classValue=TsToCompare[128]
        TsToCompare=TsToCompare[0:128]
        dfForDTree['TsIndex'].iloc[i]=i
        dfForDTree['class'].iloc[i]=classValue
        counter=0
        #scandisco e calcolo distanza dai motif
        for j in range(len(CandidatesList)):
            numMotif=len(CandidatesList['Motif'].iloc[j])
            numDiscord=len(CandidatesList['Discord'].iloc[j])
            for k in range(numMotif):
                l1=CandidatesList['Motif'].iloc[j] #lista di indice i in motifDiscordList
                startingIndex=l1[k] #indice di inizio del motif
                TsContainingCandidateShapelet = np.array(dataset.iloc[j].values) #Ts contenente candidato shapelet
                Dp=distanceProfile.massDistanceProfile(TsContainingCandidateShapelet,int(startingIndex),window_size,TsToCompare)
                minValueFromDProfile=min(Dp[0]) #Dp[0] contiene il Dp effettivo
                dfForDTree[prefix+str(counter)].iloc[i]=minValueFromDProfile
                counter+=1
            for k in range(numDiscord):
                l1=CandidatesList['Discord'].iloc[j] #lista di indice i in motifDiscordList
                startingIndex=l1[k] #indice di inizio del motif
                TsContainingCandidateShapelet = np.array(dataset.iloc[j].values) #Ts contenente candidato shapelet
                Dp=distanceProfile.massDistanceProfile(TsContainingCandidateShapelet,int(startingIndex),window_size,TsToCompare)
                minValueFromDProfile=min(Dp[0]) #Dp[0] contiene il Dp effettivo
                dfForDTree[prefix+str(counter)].iloc[i]=minValueFromDProfile
                counter+=1
        
   # print(counter)    
    return dfForDTree #columnsList2 restituito per generare poi dFrame in "Split" (struttura dframe)
                
                

In [8]:
#dataset (dframe): nella riga i: indice della ts di appartenenza, distanza tra candidato e Ts, e classe di appartenenza di Ts
#calcola entropia di un dataset basandosi sul num di classi esistenti
def computeEntropy(dataset):
    value,counts = np.unique(dataset['class'], return_counts=True)
    actualEntropy=entropy(counts, base=2)
    return actualEntropy

In [9]:
#calcola il gain tra entropia nodo padre e sommatoria entropia nodi figli (GAIN CALCOLATO SUL VALORE DELL'ATTRIBUTO)
def computeGain(entropyParent,LenDatasetParent,Dleft,Dright):
    entropyLeft=computeEntropy(Dleft)
    entropyRight=computeEntropy(Dright)
    gain=entropyParent
    summation=( ((len(Dleft)/LenDatasetParent)*entropyLeft) +  ((len(Dright)/LenDatasetParent)*entropyRight) )
    #print('entropyParent: '+str(entropyParent))
    #print('SUMMATION: '+str(summation))
    gain=gain-summation
    return gain

In [10]:
#SPLIT SLAVE
#effettua lo split del dataset sul attributo e valore fornito
def split(dataset,attribute,value): 
    columnsList=dataset.columns.values
    dizLeft=pd.DataFrame(columns=columnsList)
    dizRight=pd.DataFrame(columns=columnsList)
    for i in range(len(dataset)):
        if dataset.iloc[i][attribute] < value:
            dizLeft = dizLeft.append(dataset.iloc[i], ignore_index=True)
        else:
            dizRight = dizRight.append(dataset.iloc[i], ignore_index=True)
    return dizLeft, dizRight

In [11]:
# riceve vettore con mutual_information(gain) e in base al candidatesGroup scelto, determina il miglior attributo su cui splittare
# che non è stato ancora utilizzato
def getBestIndexAttribute(vecMutualInfo,candidatesGroup,CandidatesUsedListTrain,numberOfMotif,numberOfDiscord):
    
    #definisco gli indici dei candidati da considerare, in base al candidatesGroup scelto
    
    if(candidatesGroup==0):
        candidatesIndex=range(numberOfMotif)
    elif(candidatesGroup==1):
        candidatesIndex=range(numberOfMotif,numberOfMotif+numberOfDiscord)
    else:
        candidatesIndex=range(numberOfMotif+numberOfDiscord)
    
    #genero struttura dati associando indice candidato e gain calcolato
    
    MutualInfoDf=pd.DataFrame(columns=['candidatesIndex','gain'],index=range(len(candidatesIndex)))
    MutualInfoDf['gain']=vecMutualInfo
    MutualInfoDf['candidatesIndex']=candidatesIndex
    
    #ordino i candidati in base a gain decrescente
    
    MutualInfoDf=MutualInfoDf.sort_values(by='gain',ascending = False)

    
    #scandisco i candidati fino a trovare il candidato con miglior gain che non è ancora stato usato
    
    bestIndexAttribute=-1
    i=0
    
    #cicla fin quando trova candidato libero con gain maggiore
    while(bestIndexAttribute==-1 and i<len(MutualInfoDf)):    
        attributeToVerify=int(MutualInfoDf.iloc[i]['candidatesIndex'])
        if(CandidatesUsedListTrain.iloc[attributeToVerify]['Used']==False):
            bestIndexAttribute=attributeToVerify
            CandidatesUsedListTrain.iloc[attributeToVerify]=True #settando a true il candidato scelto, non sarà usato in seguito
            print('BEST')
            print(bestIndexAttribute)
            attributeList.append(bestIndexAttribute)
        else:
            i+=1
    
    return bestIndexAttribute
            
    
        
        

In [12]:
#SPLIT INTERMEDIO
#dato il dataset, cerca il miglior attributo e relativo valore (optimal split point) su cui splittare
# restituiendo il dataset splittato e i valori trovati
def findBestAttributeValue(dataset,candidatesGroup,CandidatesUsedListTrain,numberOfMotif,numberOfDiscord):
    #cerca e restituisce attributo migliore su cui splittaree relativo valore ottimale (optimal split point)
    #CANDIDATE GROUP permette di scegliere se usare come candidati 0=motifs 1=discord 2=entrambi
    bestGain=0
    actualGain=0
    bestvalueForSplit=0
    y = dataset['class'].values
    y=y.astype('int')
    
    #trovo best Attribute
    numAttributes=len(dataset.columns.values)
    numAttributes-=2 #tolgo i due attributi TsIndex e class dal Dframe
    datasetForMutual=pd.DataFrame()
    
    #preparo il Dframe da passare a mutual_info_classif, settando se scegliere tra motifs/discord/entrambi
    
    if(candidatesGroup==0): #solo motifs
        datasetForMutual=dataset.iloc[:,np.r_[:numberOfMotif]]
    elif(candidatesGroup==1):
        datasetForMutual=dataset.iloc[:,np.r_[numberOfMotif:numberOfMotif+numberOfDiscord]]
    else:
        datasetForMutual=dataset.iloc[:,np.r_[:numAttributes]]
        
    vecMutualInfo=mutual_info_classif(datasetForMutual, y, discrete_features=False)
    indexBestAttribute=getBestIndexAttribute(vecMutualInfo,candidatesGroup,CandidatesUsedListTrain,numberOfMotif,numberOfDiscord)
    
    

    entropyParent=computeEntropy(dataset)
    DleftChosen=pd.DataFrame()
    DrightChosen=pd.DataFrame()
    #splitto su tale attributo e trovo il valore ottimale testando su tutte le distanze calcolate tre le Ts e tale candidato 
    for i in range (len(dataset)):
        testValue=dataset.iloc[i][indexBestAttribute]
        if(testValue>0): #distanze non-negative, split inutile su tale valore
            Dleft,Dright=split(dataset,indexBestAttribute,testValue)
            actualGain=computeGain(entropyParent,len(dataset),Dleft,Dright)
            if(actualGain > bestGain):
                bestGain=actualGain
                bestvalueForSplit=testValue
                DleftChosen=Dleft
                DrightChosen=Dright

    
    return indexBestAttribute,bestvalueForSplit,DleftChosen,DrightChosen

In [13]:
#SPLIT MASTER
# funzione ricorsiva che implementa la creazione dell'albero di classificazione
# memorizza in ogni nodo: attributo, valore attributo su cui splitto, entropia nodo, num pattern
# memorizza in ogni foglia: entropia nodo, num pattern, classe nodo

# VERSIONE CHE RIMUOVE I CANDIDATI QUANDO VENGONO SCELTI 

def buildTree(actualNode,dataset,maxDepth, minSamplesLeaf, depth,candidatesGroup,CandidatesUsedListTrain,numberOfMotif,numberOfDiscord):
    #caso base: num pattern < soglia minima || profondità massima raggiunta => genero foglia con media delle classi
    #DATASET HA SEMPRE ALMENO UN PATTERN
    boolValue=checkIfIsLeaf(dataset)
    if(len(dataset)<minSamplesLeaf or depth>=maxDepth or boolValue==True ):
        average = sum(dataset['class'].values) / len(dataset['class'].values)
        classValue = round(average)
        numPattern=len(dataset)
        entropy=computeEntropy(dataset)
        
        nodeInfo=list()
        nodeInfo.append(classValue)
        nodeInfo.append(numPattern)
        nodeInfo.append(entropy)
    
        actualNode.data=nodeInfo
        actualNode.value=-1
        actualNode.left=None
        actualNode.right=None
        #print(dataset['class'])
        return 
    #caso ricorsivo in cui si può splittare
    else:
        
        indexChosenAttribute,attributeValue,Dleft,Dright=findBestAttributeValue(dataset,candidatesGroup,CandidatesUsedListTrain,numberOfMotif,numberOfDiscord)
        numPattern=len(dataset)
        entropy=computeEntropy(dataset)
        
        #memorizzo nel nodo l'attributo, il valore e altre info ottenute dallo split
        
        nodeInfo=list()
        nodeInfo.append(attributeValue)
        nodeInfo.append(numPattern)
        nodeInfo.append(entropy)
        actualNode.data=nodeInfo
        actualNode.value=(indexChosenAttribute)
        
        #se possibile richiamo ricorsivamente sul nodo dx e sx figlio
        if(len(Dleft)>0):
            actualNode.left=Node(indexChosenAttribute)
            buildTree(actualNode.left,Dleft,maxDepth, minSamplesLeaf, depth+1,candidatesGroup,CandidatesUsedListTrain,numberOfMotif,numberOfDiscord)
        
        if(len(Dright)>0):
            actualNode.right=Node(indexChosenAttribute)
            buildTree(actualNode.right,Dright,maxDepth, minSamplesLeaf, depth+1,candidatesGroup,CandidatesUsedListTrain,numberOfMotif,numberOfDiscord)
            
        

In [14]:
#verifica se dataset, ha pattern appartenenti ad una sola classe => è gia foglia
def checkIfIsLeaf(dataset):
    isLeaf=True
    entropy=computeEntropy(dataset)
    if(entropy>0):
        isLeaf=False
    return isLeaf

In [15]:
#effettua il primo passo dell'algo di generazione dell'albero, richiama ricorsivamente sui figli
# VERSIONE CHE NON RIMUOVE I CANDIDATI QUANDO VENGONO SCELTI 
def startAlgo(dfForDTree,candidatesGroup,CandidatesUsedListTrain,maxDepth,minSamplesLeaf,numberOfMotif,numberOfDiscord):
    
    #inizio algo per nodo radice
    indexChosenAttribute,attributeValue,Dleft,Dright=findBestAttributeValue(dfForDTree,candidatesGroup,CandidatesUsedListTrain,numberOfMotif,numberOfDiscord)
    root=Node(indexChosenAttribute)
    numPattern=len(dfForDTree)
    entropy=computeEntropy(dfForDTree)
        
    #memorizzo nel nodo l'attributo, il valore e altre info ottenute dallo split
        
    nodeInfo=list()
    nodeInfo.append(attributeValue)
    nodeInfo.append(numPattern)
    nodeInfo.append(entropy)
    root.data=nodeInfo
    
    root.left=Node(indexChosenAttribute)
    root.right=Node(indexChosenAttribute)
    
    #chiamata ricorsiva
    if(len(Dleft)>0):
        buildTree(root.left,Dleft,maxDepth, minSamplesLeaf,1,candidatesGroup,CandidatesUsedListTrain,numberOfMotif,numberOfDiscord)
    if(len(Dright)>0):
        buildTree(root.right,Dright,maxDepth, minSamplesLeaf, 1,candidatesGroup,CandidatesUsedListTrain,numberOfMotif,numberOfDiscord)
    return root

In [16]:
#stampa dell'albero
def printAll(Root):
    if(Root.left==None and Root.right==None):
        print('foglia')
    print('Nodo: '+str(Root.value))
    df=Root.data
    print(df)
    print("\n")
    if(Root.left!=None):
        printAll(Root.left)
    if(Root.right!=None):
        printAll(Root.right)

In [17]:
def predict(testDataset,root):
    print('START PRED')
    #preparo dataset 
    numAttributes=len(testDataset.columns.values)
    numAttributes-=2 #per prendere solo gli attributi utili a xTest
    yTest=testDataset.iloc[:]['class'].values
    yPredicted=np.zeros( len(yTest) )
    xTest=testDataset.iloc[:,np.r_[:numAttributes]]
    
    #effettuo predizione per ogni pattern
    
    for i in range(len(xTest)):
        pattern=xTest.iloc[i]
        yPredicted[i]= treeExplorer(pattern,root)
    
    yTest = yTest.astype(int)
    yPredicted = yPredicted.astype(int)
    
    return yTest,yPredicted
        

In [18]:
def treeExplorer(pattern,node):
    #caso base, node è foglia
    if(node.value==-1):
        return int(node.data[0])
    else:
    #caso ricorsivo
        attr='cand'+str(node.value)
        if(pattern[attr] < node.data[0]):
            return treeExplorer(pattern,node.left)
        else:
            return treeExplorer(pattern,node.right)

In [49]:
def computeSubSeqDistanceForTest(datasetTest,datasetTrain,attributeList,CandidatesList,numberOfMotif,numberOfDiscord):
    #quantifico il num di candidati usati dall'albero e in base a tale valore genero colonne per dfForDTree
      #quantifico il num di candidati e in base a tale valore genero colonne per dfForDTree
    columnsList2=list()
    prefix='cand'
    for i in attributeList:
        columnsList2.append(prefix+str(i)) 
    columnsList2.append('TsIndex')
    columnsList2.append('class')
    dfForDTree=pd.DataFrame(columns=columnsList2,index=range(0,len(datasetTest)))

    #per ogni Ts, scandisco ogni candidato e calcolo la distanza minore 
    for i in range(len(datasetTest)):
        #acquisisco la Ts
        TsToCompare = np.array(datasetTest.iloc[i].values) 
        classValue=TsToCompare[128]
        TsToCompare=TsToCompare[0:128]
        dfForDTree['TsIndex'].iloc[i]=i
        dfForDTree['class'].iloc[i]=classValue
        counter=0
        #scandisco e calcolo distanza dai motif
        for z in range(len(attributeList)):
            candidateIndex=attributeList[z]
            counter=0
            for j in range(len(CandidatesList)):
                numMotif=len(CandidatesList['Motif'].iloc[j])
                numDiscord=len(CandidatesList['Discord'].iloc[j])
                for k in range(numMotif):
                    if(counter==candidateIndex):
                        l1=CandidatesList['Motif'].iloc[j] #lista di indice i in motifDiscordList
                        startingIndex=l1[k] #indice di inizio del motif
                        TsContainingCandidateShapelet = np.array(datasetTrain.iloc[j].values) #Ts contenente candidato shapelet
                        Dp=distanceProfile.massDistanceProfile(TsContainingCandidateShapelet,int(startingIndex),window_size,TsToCompare)
                        minValueFromDProfile=min(Dp[0]) #Dp[0] contiene il Dp effettivo
                        print(counter,candidateIndex,minValueFromDProfile)
                        dfForDTree[prefix+str(counter)].iloc[i]=minValueFromDProfile
                    counter+=1
                        
                for k in range(numDiscord):
                    if(counter==candidateIndex):
                        print(counter,candidateIndex)
                        l1=CandidatesList['Discord'].iloc[j] #lista di indice i in motifDiscordList
                        startingIndex=l1[k] #indice di inizio del motif
                        TsContainingCandidateShapelet = np.array(datasetTrain.iloc[j].values) #Ts contenente candidato shapelet
                        Dp=distanceProfile.massDistanceProfile(TsContainingCandidateShapelet,int(startingIndex),window_size,TsToCompare)
                        minValueFromDProfile=min(Dp[0]) #Dp[0] contiene il Dp effettivo
                        print(counter,candidateIndex,minValueFromDProfile)
                        dfForDTree[prefix+str(counter)].iloc[i]=minValueFromDProfile
                    counter+=1
                    



    le = LabelEncoder()        
    num_classes = le.fit_transform(dfForDTree['class'])
    dfForDTree['class']=num_classes
            
    return dfForDTree #columnsList2 restituito per generare poi dFrame in "Split" (struttura dframe)
                
    
    #print(CandidatesList)
    #print(attributeDF)
    

In [41]:
#ACQUISISCO STRUTTURE DATI DEL TRAINING SET
dataset = arff.loadarff('CBF/CBF_TRAIN.arff')
dfTrain=pd.DataFrame(dataset[0])
window_size=5
mpTrain,CandidatesListTrain,numberOfMotifTrain,numberOfDiscordTrain,CandidatesUsedListTrain=getDataStructures(dfTrain,window_size)
dfForDTree=computeSubSeqDistance(dfTrain,CandidatesListTrain)


Candidati estratti
            Motif    Discord
0     [9.0, 26.0]   [61, 27]
1   [105.0, 54.0]   [8, 119]
2    [17.0, 40.0]    [31, 1]
3    [48.0, 12.0]   [23, 86]
4          [27.0]   [38, 62]
5    [45.0, 13.0]  [119, 26]
6     [7.0, 37.0]   [98, 76]
7     [2.0, 24.0]    [36, 8]
8     [1.0, 29.0]   [23, 32]
9     [50.0, 0.0]   [38, 59]
10   [11.0, 17.0]   [80, 23]
11    [31.0, 7.0]  [114, 54]
12         [53.0]   [61, 10]
13    [79.0, 6.0]  [105, 38]
14         [59.0]    [98, 8]
15    [2.0, 45.0]     [1, 8]
16    [28.0, 3.0]    [5, 88]
17   [14.0, 35.0]  [118, 85]
18    [13.0, 3.0]   [111, 8]
19    [19.0, 6.0]   [92, 35]
20   [19.0, 39.0]   [26, 70]
21         [26.0]   [13, 35]
22   [18.0, 22.0]   [59, 25]
23   [27.0, 42.0]  [32, 104]
24    [8.0, 28.0]    [4, 95]
25         [16.0]   [87, 20]
26   [67.0, 59.0]   [33, 83]
27   [16.0, 39.0]   [75, 99]
28   [11.0, 41.0]   [47, 39]
29    [6.0, 15.0]  [117, 51]
55 60
      Used
0    False
1    False
2    False
3    False
4    False
..     ...

In [53]:
#COSTRUISCO DECISION TREE
candidatesGroup=1
albero=None
maxDepth=3
minSamplesLeaf=5
attributeList=list()
albero=startAlgo(dfForDTree,candidatesGroup,CandidatesUsedListTrain,maxDepth,minSamplesLeaf,numberOfMotifTrain,numberOfDiscordTrain)
print(albero)
print(attributeList)
printAll(albero)



BEST
102
BEST
57
BEST
96
BEST
90
BEST
74

           __________102___
          /                \
     ____57___             _74
    /         \           /   \
  _96         _90        -1    -1
 /   \       /   \
-1    -1    -1    -1

[102, 57, 96, 90, 74]
Nodo: 102
[0.8272897225955013, 30, 1.565596230357602]


Nodo: 57
[0.8411856192913642, 21, 1.509968612636817]


Nodo: 96
[0.7946792550073827, 14, 1.3787834934861756]


foglia
Nodo: -1
[2.0, 9, 0.5032583347756457]


foglia
Nodo: -1
[1.0, 5, 0.9709505944546688]


Nodo: 90
[0.8036776280851692, 7, 0.8631205685666311]


foglia
Nodo: -1
[1.0, 3, 0.0]


foglia
Nodo: -1
[0.0, 4, 1.0]


Nodo: 74
[0.3999921693358199, 9, 0.9182958340544894]


foglia
Nodo: -1
[1.0, 3, 0.0]


foglia
Nodo: -1
[0.0, 6, 0.0]




[55, 69, 74, 76, 103, 110]


In [55]:
#GENERO STRUTTURE DATI PER TEST SET
dataset2 = arff.loadarff('CBF/CBF_TEST.arff')
window_size=5
dfTest = pd.DataFrame(dataset2[0]) #30 record su matrice da 128 attributi + 'b': classe appartenenza  
dfTest=dfTest.iloc[:50] #ne prendo 50 altrimenti impiega tempo troppo lungo, sono 900 record totali

attributeList=sorted(attributeList)
dfForDTreeTest=computeSubSeqDistanceForTest(dfTest,dfTrain,attributeList,CandidatesListTrain,numberOfMotifTrain,numberOfDiscordTrain) 
print(dfForDTreeTest)

#CONTROLLA, DOVRRBBE ESSERE GIUSTO COSI, SE USO COMPUTE NORMALE CON dfTest SBAGLIO DATASET DI RIFERIMENTO 


57 57 0.809992685252394
74 74 0.7534794388821523
90 90
90 90 0.650691353468949
96 96 0.42694682207756474
102 102
102 102 1.1559961182500147
57 57 0.9122636784863627
74 74 0.78690178847362
90 90
90 90 0.7930835496101084
96 96 1.0461235556231032
102 102
102 102 0.5269193515356734
57 57 0.7617678365167475
74 74 0.6142363602539986
90 90
90 90 0.7113268719494577
96 96 0.8610746434383632
102 102
102 102 0.9216260390067349
57 57 0.9856118759110345
74 74 0.8466361659082225
90 90
90 90 0.5226683701107118
96 96 0.6531495668270005
102 102
102 102 0.5875242844193144
57 57 0.9157737545133299
74 74 0.6112377383034681
90 90
90 90 0.4106400490785593
96 96 0.5523589645991127
102 102
102 102 0.5531529629431354
57 57 0.30390097761055473
74 74 0.7914843956167111
90 90
90 90 0.7619181798239009
96 96 0.5295635864746313
102 102
102 102 0.5533754883556045
57 57 0.8118618039634661
74 74 0.6357515464842131
90 90
90 90 1.1991052807593923
96 96 0.7139260767309714
102 102
102 102 0.5303639698565529
57 57 0.4090403

In [56]:
#EFFETTUO PREDIZIONE E MISURO RISULTATO

yTest,yPredicted=predict(dfForDTreeTest,albero)



for a,b in zip(yTest,yPredicted):
    print(a,b)
    
print(type(yPredicted))
print(type(yTest))
    
print(classification_report(yTest, yPredicted))
print('Accuracy %s' % accuracy_score(yTest, yPredicted))
print('F1-score %s' % f1_score(yTest, yPredicted, average=None))
confusion_matrix(yTest, yPredicted)    

START PRED
1 0
1 1
0 0
1 1
1 1
2 2
0 2
2 2
1 2
2 1
1 2
0 0
2 0
1 2
1 1
2 2
2 0
1 1
1 1
0 0
1 0
1 1
1 1
0 0
1 2
2 2
2 2
2 2
0 0
2 1
1 2
1 0
0 2
2 2
1 2
2 0
0 1
2 2
2 1
2 1
1 1
1 1
1 0
1 2
1 2
0 2
1 2
2 2
0 2
1 1
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
              precision    recall  f1-score   support

           0       0.42      0.50      0.45        10
           1       0.69      0.46      0.55        24
           2       0.41      0.56      0.47        16

    accuracy                           0.50        50
   macro avg       0.50      0.51      0.49        50
weighted avg       0.54      0.50      0.51        50

Accuracy 0.5
F1-score [0.45454545 0.55       0.47368421]


array([[ 5,  1,  4],
       [ 4, 11,  9],
       [ 3,  4,  9]], dtype=int64)